In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
#working with simple data ,(no preprocessing done now),suppose these are the 4 documemnts
textFiles = ["Doc1.txt","Doc2.txt","Doc3.txt"]

In [50]:
import nltk
import string
from nltk.stem.porter import PorterStemmer
import codecs

token_dict = {}
stemmer = PorterStemmer()
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens, stemmer)
    return stems

for file_name in textFiles:
    with codecs.open(file_name, "r",encoding='utf-8', errors='ignore') as f:  
        text = f.read().encode('utf-8').strip()
        text = text.replace('\r', ' ')
        text = text.replace('\n', ' ')
        lowers = text.lower().strip()
        no_punctuation = lowers.translate(None, string.punctuation)
        token_dict[file_name] = no_punctuation

In [49]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')
tfs = tfidf.fit_transform(token_dict.values()).toarray()
feature_names = tfidf.get_feature_names()
for col in tfs.nonzero()[1]:
    print feature_names[col], ' - ', tfs[0, col]


 0005  -  0.0295822643781
0006  -  0.0224980745558
0007  -  0.0174717618868
0008  -  0.0174717618868
0009  -  0.0224980745558
0010  -  0.0224980745558
1  -  0.0295822643781
10  -  0.0224980745558
20  -  0.0674942236675
20090257945  -  0.0295822643781
30  -  0.236658115025
40  -  0.0591645287562
50  -  0.0591645287562
60  -  0.0591645287562
70  -  0.0295822643781
a1  -  0.0295822643781
ablat  -  0.0295822643781
achiev  -  0.0295822643781
addit  -  0.0349435237737
address  -  0.0174717618868
aforement  -  0.0174717618868
align  -  0.0698870475474
applic  -  0.0349435237737
arc  -  0.0224980745558
argon  -  0.0295822643781
art  -  0.0174717618868
assembl  -  0.0295822643781
atom  -  0.0591645287562
attain  -  0.0295822643781
base  -  0.0224980745558
becaus  -  0.0224980745558
bed  -  0.0295822643781
befor  -  0.0591645287562
candid  -  0.0295822643781
carbon  -  0.192189380755
carrier  -  0.0591645287562
catalyst  -  0.202482671002
catalyt  -  0.147911321891
caus  -  0.0295822643781
ccvd 

In [51]:
from sklearn.metrics.pairwise import cosine_similarity


In [55]:
#finding cosine similarity with doc1 with other documents
cosine_similarities = cosine_similarity(tfs[1:2], tfs)



In [56]:
cosine_similarities

array([[ 0.2322161,  1.       ,  0.1548725]])